## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,County,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hachinohe,JP,40.5000,141.5000,68.63,58,82,10.60,broken clouds
1,1,Bandarbeyla,SO,9.4942,50.8122,82.69,61,61,25.68,broken clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,35.44,64,0,10.36,clear sky
3,3,Vardo,NO,70.3705,31.1107,35.80,66,55,9.98,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,46.89,82,0,3.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,County,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bandarbeyla,SO,9.4942,50.8122,82.69,61,61,25.68,broken clouds
12,12,Hilo,US,19.7297,-155.0900,76.28,86,90,4.61,light rain
22,22,Butaritari,KI,3.0707,172.7902,82.44,75,47,15.43,moderate rain
24,24,Dingle,PH,10.9995,122.6711,76.59,93,100,6.44,light rain
28,28,Bundaberg,AU,-24.8500,152.3500,75.49,13,0,10.36,clear sky
34,34,Tiznit,MA,29.5833,-9.5000,75.65,37,31,1.63,scattered clouds
41,41,Gunjur,GM,13.2019,-16.7339,80.67,94,40,0.00,scattered clouds
42,42,Avera,US,33.1940,-82.5271,76.51,69,100,1.05,overcast clouds
45,45,Hengshui,CN,37.7322,115.7011,77.86,57,0,9.64,clear sky
47,47,Bengkulu,ID,-3.8004,102.2655,82.00,73,45,9.62,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
County                 218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                218
City                   218
County                 218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "County", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,County,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bandarbeyla,SO,82.69,broken clouds,9.4942,50.8122,
12,Hilo,US,76.28,light rain,19.7297,-155.0900,
22,Butaritari,KI,82.44,moderate rain,3.0707,172.7902,
24,Dingle,PH,76.59,light rain,10.9995,122.6711,
28,Bundaberg,AU,75.49,clear sky,-24.8500,152.3500,
34,Tiznit,MA,75.65,scattered clouds,29.5833,-9.5000,
41,Gunjur,GM,80.67,scattered clouds,13.2019,-16.7339,
42,Avera,US,76.51,overcast clouds,33.1940,-82.5271,
45,Hengshui,CN,77.86,clear sky,37.7322,115.7011,
47,Bengkulu,ID,82.00,scattered clouds,-3.8004,102.2655,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=["Hotel Name"], inplace = True)
hotel_df

,City,County,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bandarbeyla,SO,82.69,broken clouds,9.4942,50.8122,JABIR HOTEL
12,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Butaritari,KI,82.44,moderate rain,3.0707,172.7902,Isles Sunset Lodge
24,Dingle,PH,76.59,light rain,10.9995,122.6711,SEGAYA PROPERTY
28,Bundaberg,AU,75.49,clear sky,-24.8500,152.3500,Takalvan Motel
...,...,...,...,...,...,...,...
664,San Juan De Abajo,MX,75.67,overcast clouds,20.8167,-105.1833,Lorenzo Anaya
668,Singapore,SG,86.04,broken clouds,1.2897,103.8501,The Fullerton Hotel Singapore
678,Kawalu,ID,80.85,broken clouds,-7.3817,108.2082,KOSAN UNGU UNSIK
680,Erdemli,TR,80.58,clear sky,36.6050,34.3084,Sun Rice Otel


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{County}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))